In [1]:
import os

os.chdir('../')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data_train_path = "Dataset/data_train.csv"

URM_all_dataframe = pd.read_csv(data_train_path)
print(URM_all_dataframe.head(10))

   user_id  item_id  data
0        0        0   1.0
1        0        2   1.0
2        0      120   1.0
3        0      128   1.0
4        0      211   1.0
5        0      232   1.0
6        0      282   1.0
7        0      453   1.0
8        0      458   1.0
9        0      491   1.0


In [3]:
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout



URM_all = sps.coo_matrix((URM_all_dataframe['data'], (URM_all_dataframe['user_id'], URM_all_dataframe['item_id'])))
URM_all = URM_all.tocsr()

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions


In [ ]:
# too slow -> from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.ImplicitALSRecommender import ImplicitALSRecommender # Credit Nicola Cesare Github
import optuna

def objective_function(trial):
    
    '''recommender_implicitAls.fit(iterations= int(83.64347328971459), factors= int(125.3856187337561), regularization= 0.05106206327352681, use_gpu=False, num_threads=0, confidence_scaling=linear_scaling_confidence,**{"alpha":5.579621138255679})
'''
    
    factors = trial.suggest_int("factors", 1, 1000)
    iterations = trial.suggest_int("iterations", 1, 1000)
    regularization = trial.suggest_uniform("regularization", 0.0, 1.0)
    alpha = trial.suggest_uniform("alpha", 0.0, 50.0)
    
    
    recommender = ImplicitALSRecommender(URM_train_validation)
    recommender.fit(iterations=iterations, factors=factors, regularization=regularization, use_gpu=False, num_threads=0, confidence_scaling="linear", **{"alpha":alpha})

    result_dict, _ = evaluator_test.evaluateRecommender(recommender)

    MAP = result_dict.loc[10]["MAP"]
    
    return MAP


class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame(columns = ["result"])
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        self.results_df = pd.concat([self.results_df, pd.DataFrame([hyperparam_dict])], ignore_index=True)
        self.results_df.to_csv("logs/IALS/IALS_first_trial.csv", index = False)

optuna_study = optuna.create_study(study_name="IALS", direction="maximize")
        
save_results = SaveResults()
        
optuna_study.optimize(objective_function,
                      callbacks=[save_results],
                      n_trials = 100
                      )

[I 2024-12-27 17:13:02,511] A new study created in memory with name: IALS
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_72512/1197196737.py:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  regularization = trial.suggest_uniform("regularization", 0.0, 1.0)
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_72512/1197196737.py:13: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform("alpha", 0.0, 50.0)
/Users/lucapagano/miniconda3/envs/RecSysPD/lib/python3.8/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 8 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THR

  0%|          | 0/38 [00:00<?, ?it/s]

EvaluatorHoldout: Processed 35595 (100.0%) in 20.09 sec. Users per second: 1772


[I 2024-12-27 17:15:08,785] Trial 0 finished with value: 0.030913419710721556 and parameters: {'factors': 199, 'iterations': 38, 'regularization': 0.0012202016324741605, 'alpha': 36.767148317560746}. Best is trial 0 with value: 0.030913419710721556.
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_72512/1197196737.py:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  regularization = trial.suggest_uniform("regularization", 0.0, 1.0)
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_72512/1197196737.py:13: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform("alpha", 0.0, 50.0)


  0%|          | 0/875 [00:00<?, ?it/s]